In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tyssue
import pandas as pd
import numpy as np
import json

In [3]:
from tyssue.core import Epithelium

In [4]:
with pd.get_store('before_apoptosis.hf5') as store:
    cell_df = store.get('cell_df')
    jv_df = store.get('jv_df')
    je_df = store.get('je_df')


In [5]:
eptm = Epithelium('emin', cell_df, jv_df, je_df)

In [6]:
eptm.cell_df.head()

,z,area,is_alive,perimeter,x,y,old_idx
cell,,,,,,,
0,-46.924275,12.000000,0,0.000000,26.662337,1.987162,0
1,-45.602975,9.791406,1,12.335801,27.251027,-0.223783,1
2,-41.930102,6.465672,1,10.006895,28.236650,0.707790,2
3,-40.636328,8.709595,1,11.041760,28.149745,-2.142625,3
4,-38.791643,8.197723,1,10.799216,27.901080,3.809064,4


we define the adimentional contractility $\bar\Gamma = \Gamma/K_vA_0h_0^2$ and line tension
$\bar\Lambda = \Lambda /K_v (A_0^{3/2}h_0^2)$, where $h_0$ is such that $V_0 = A_0h_0$.


In [7]:
parameters = {'contractility': 0.04,
              'line_tension': 0.12,
              'prefered_area':10.,
              'prefered_height':24.,
              'vol_elasticity':1.,
              'rho_lumen':4.}
with open('default_params.json', 'w') as param_json:
    json.dump(parameters, param_json)

def dimentionalize(parameters):
    dim_params = parameters.copy()
    Kv = parameters['vol_elasticity']
    A0 = parameters['prefered_area']
    h0 = parameters['prefered_height']
    
    dim_params['contractility'] = parameters['contractility'] * (Kv * A0 * h0**2)
    dim_params['line_tension'] = parameters['line_tension'] * (Kv * A0**1.5 * h0**2)
    dim_params['prefered_vol'] = A0 * h0
    return dim_params

parameters = dimentionalize(parameters)



In [8]:
for key in ['vol_elasticity', 'prefered_area', 'prefered_vol',
            'prefered_height', 'contractility']:
    eptm.cell_df[key] = parameters[key]

eptm.je_df['line_tension'] = parameters['line_tension']


In [9]:
eptm.je_df.head()
for nc in ['nx', 'ny', 'nz']:
    eptm.je_df[nc] = 0

In [10]:
from tyssue.geometry import sheet_geometry

In [11]:
sheet_geometry.update_all(eptm)

In [12]:
def update_vol(eptm):
    eptm.cell_df['height'] = np.hypot(eptm.cell_df['y'],
                                      eptm.cell_df['x']) - parameters['rho_lumen']
    eptm.cell_df['vol'] = eptm.cell_df['height'] * eptm.cell_df['area']

update_vol(eptm)

def compute_energy(eptm, full_output=False):

    E_t = eptm.je_df['line_tension'] * eptm.je_df['length']


    E_v =  0.5 * (eptm.cell_df['vol_elasticity']
                  * (eptm.cell_df['vol']
                     - eptm.cell_df['prefered_vol'])**2
                  ) * eptm.cell_df['is_alive']
    E_c = 0.5 * (eptm.cell_df['contractility']
                 * eptm.cell_df['perimeter']**2
                 ) * eptm.cell_df['is_alive']
    if full_output:
        return E_t, E_c, E_v
    else:
        return E_t.sum() + (E_c+E_v).sum()

compute_energy(eptm, full_output=False)

71126993.64768562

In [13]:
eptm.dcoords = ['dx', 'dy', 'dz']

In [14]:
eptm.je_df.head()

,,,length,dz,dy,dx,old_jv0,old_jv1,old_cell,line_tension,nx,ny,nz
srce,trgt,cell,,,,,,,,,,,
1,0,0,2.491670,-0.665097,-2.382007,-0.303487,928,929,0,2185.766319,1.918098e+00,8.877496e-02,-1.193180e+00
2,1,0,2.639199,0.951623,-2.359688,0.701183,929,930,0,2185.766319,-8.437695e-15,-2.498002e-15,5.551115e-17
0,1,1,2.491670,0.665097,2.382007,0.303487,928,929,1,2185.766319,3.339108e+00,-6.864496e-02,-1.277804e+00
3,0,1,2.674664,-1.356183,2.278370,-0.351600,928,931,1,2185.766319,3.519823e+00,-2.468454e-01,-1.327235e+00
1,6,1,1.039101,0.938752,0.093970,0.435481,929,934,1,2185.766319,2.156034e+00,-1.024630e-01,-9.899148e-01


In [15]:
def _to_3d(df):

    df_3d = np.asarray(df).repeat(3).reshape((df.size, 3))
    return df_3d

def compute_gradient(eptm, components=False):
    '''
    If components is True, returns the individual terms
    (grad_t, grad_c, grad_v)
    '''
    
    eptm.grad_i_lij = - (eptm.je_df[eptm.dcoords]
                         / _to_3d(eptm.je_df['length']))


    grad_t = tension_grad(eptm)
    grad_c = contractile_grad(eptm)
    grad_v = volume_grad(eptm)

    grad_i = grad_t + grad_c + grad_v
    if components:
        return grad_i, grad_t, grad_c, grad_v
    return grad_i


def tension_grad(mesh):

    _grad_t = eptm.grad_i_lij * _to_3d(eptm.je_df['line_tension'])
    grad_t = _grad_t.sum(level='srce').loc(eptm.jv_idx)

    return grad_t


def contractile_grad(mesh):

    contract = eptm.cell_df['contractility']
    perimeter = eptm.cell_df['perimeter']

    gamma_L = contract * perimeter
    gamma_L = eptm.upcast_cell(df=gamma_L)

    _grad_c = eptm.grad_i_lij * _to_3d(gamma_L)
    grad_c = _grad_c.sum(level='srce').loc(eptm.jv_idx)

    return grad_c


def volume_grad(mesh):
    '''
    Computes :math:`\sum_\alpha\nabla_i \left(K (V_\alpha - V_0)^2\right)`
    '''

    elasticity = eptm.cell_df['vol_elasticity']
    pref_V = eptm.cell_df['prefered_vol']
    V = eptm.cell_df['vol']
    KV_V0 = elasticity * (V - pref_V)
    tri_KV_V0 = eptm.upcast_cell(df=KV_V0)

    r_ijs = eptm.je_df[eptm.dcoords]
    cross_ur = pd.DataFrame(np.cross(eptm.je_df[eptm.normal_coords], r_ijs),
                            columns=eptm.coords)

    h_nu = eptm.cell_df['height'] / (2 * eptm.cell_df['num_sides'])

    grad_i_V_cell = cross_ur.sum(level='cell') * _to_3d(KV_V0 * h_nu)

    cell_term = eptm.upcast_cell(df=grad_i_V_cell)
    
    grad_v = cell_term.sum(level='srce') / 2
    
    r_to_rho = eptm.jv_df[eptm.coords] / _to_3d(eptm.jv_df['rho'])
    r_to_rho = eptm.upcast_srce(df=r_to_rho)
    
    r_ai = eptm.je_df[eptm.dcoords]
    normals = eptm.je_df[mesh.normal_coords]
    cross_aj = pd.DataFrame(np.cross(normals, -r_ai),
                            columns=eptm.coords)

    tri_height = eptm.upcast_cell(columns=['height'])

    sub_area = eptm.je_df['sub_area']
    
    

    ij_term = _to_3d(tri_KV_V0) * (_to_3d(sub_area / 2) * r_to_rho -
                                    _to_3d(tri_height / 2) * cross_aj)
    grad_v += ij_term.sum(level='srce')
    return grad_v


In [16]:
grad_i, grad_t, grad_c, grad_v = compute_gradient(eptm, components=True)

TypeError: upcast_cell() got an unexpected keyword argument 'df'